In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fasttext
import bz2
import csv
from sklearn.metrics import roc_auc_score
import os
print(os.listdir("."))

['archive.zip', 'fasttext.ipynb', 'model.bin', 'sentiment.py', 'test.ft.txt.bz2', 'train.ft.txt.bz2', '__init__.py', '__pycache__']


In [2]:
data = bz2.BZ2File("./train.ft.txt.bz2")
data = data.readlines()
data = [x.decode('utf-8') for x in data]
print(len(data))

3600000


In [3]:
data[1:5]

["__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.\n",
 '__label__2 Amazing!: This soundtrack is my favorite music of all time, hands down. The intense sadness of "Prisoners of Fate" (which means all the more if you\'ve played the game) and the hope in "A Distant Promise" and "Girl who Stole the Star" have been an important inspiration to me personally throughout my teen years. The higher energy tracks like "Chrono Cross ~ Time\'s Scar~", "Time of the Dreamwatch", and "Chronomantique" (indefinably remeniscent of Chrono Tri

In [7]:
data = pd.DataFrame(data)
data.to_csv("train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

# Modelling
# This routine takes about 5 to 10 minutes
model = fasttext.train_supervised('train.txt',label_prefix='__label__', thread=4, epoch = 10)
print(model.labels, 'are the labels or targets the model is predicting')

['__label__1', '__label__2'] are the labels or targets the model is predicting


In [4]:
data = pd.DataFrame(data)
data.to_csv("train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

model = fasttext.load_model('model.bin')
print(model.labels, 'are the labels or targets the model is predicting')

['__label__1', '__label__2'] are the labels or targets the model is predicting


In [5]:
test = bz2.BZ2File("./test.ft.txt.bz2")
test = test.readlines()
test = [x.decode('utf-8') for x in test]
print(len(test), 'number of records in the test set')

# To run the predict function, we need to remove the __label__1 and __label__2 from the testset.
new = [w.replace('__label__2 ', '') for w in test]
new = [w.replace('__label__1 ', '') for w in new]
new = [w.replace('\n', '') for w in new]

# Use the predict function
pred = model.predict(new)

# check the first record outputs
print(pred[0][0], 'is the predicted label')
print(pred[0][1], 'is the probability score')

400000 number of records in the test set
['__label__2'] is the predicted label
['__label__2'] is the probability score


In [8]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, PrecisionRecallDisplay, precision_recall_curve
import matplotlib.pyplot as plt

# Lets recode the actual targets to 1's and 0's from both the test set and the actual predictions
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test]
pred_labels = [0 if x == ['__label__1'] else 1 for x in pred[0]]

# run the accuracy measure.
print("AUC Score: ", roc_auc_score(labels, pred_labels))
print("F1 Score: ", f1_score(labels, pred_labels))
print("Accuracy Score: ", accuracy_score(labels, pred_labels))

# precision, recall, _ = precision_recall_curve(labels, pred_labels)
# disp = PrecisionRecallDisplay(precision=precision, recall=recall)
# disp.plot()

# display = PrecisionRecallDisplay.from_estimator(
#     model, new, labels, name="FastText", plot_chance_level=True
# )
# _ = display.ax_.set_title("2-class Precision-Recall curve")

AUC Score:  0.9173375000000001
F1 Score:  0.9173951169059581
Accuracy Score:  0.9173375


TypeError: <fasttext.FastText._FastText object at 0x0000014FE3A3ED60> is not an estimator instance.

In [11]:
model.save_model("./model.bin")